In [1]:
def compare_strings(str1, str2):
    # Split the strings into words
    words1 = str1.split()
    words2 = str2.split()

    # Initialize a counter for differences and a list for the different words
    diff_count = 0
    diff_words = []

    # Get the length of the shorter list to avoid index errors
    length = min(len(words1), len(words2))

    # Compare words at the same positions
    for i in range(length):
        if words1[i] != words2[i]:
            diff_count += 1
            diff_words.append((words1[i], words2[i]))

    # Any extra words in one string are also considered as differences
    diff_count += abs(len(words1) - len(words2))
    diff_words.extend((word, '') for word in words1[length:])
    diff_words.extend(('', word) for word in words2[length:])

    return diff_count, diff_words

# # Test the function
# str1 = "I love to code in Python"
# str2 = "I love to swim in Python"

# print(compare_strings(str1, str2))  # Output: (1, [('code', 'swim')])


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
model = AutoModelForSeq2SeqLM.from_pretrained(
    "/home/dezs/Projects/Nienluan2/ModelStep8"
)
# model = AutoModelForSeq2SeqLM.from_pretrained('/home/dezs/Projects/Nienluan2/ModelStep1')
# model = AutoModelForSeq2SeqLM.from_pretrained('/home/dezs/Projects/Nienluan2/myModelMini')
model.to("cuda")




T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
# import pandas as pd
# from predict2 import predict

# def print_columns(csv_filename):
#     # Read the CSV file
#     df = pd.read_csv(csv_filename)
#     # Loop through each row and print the data in the first and second columns
#     for index, row in df.iterrows():
#         prediction = predict(model, row['AccentlessSentences'])
#         diff_count_1, diff_words_1 = compare_strings(row['AccentlessSentences'], row['Sentences'])
#         diff_count_2, diff_words_2 = compare_strings(prediction, row['Sentences'])
#         accuracy = (diff_count_1 - diff_count_2) / diff_count_1
#         if accuracy < 0:
#             accuracy=-1
        
#         print('Accuracy=', accuracy) 
#         print(diff_count_1, diff_words_1)
#         print(diff_count_2, diff_words_2)

# # Call the function with your CSV file
# print_columns('myDataset/testDataset.csv')

import pandas as pd
from predict2 import predict

def print_columns(csv_filename):
    # Read the CSV file
    df = pd.read_csv(csv_filename)
    
    accuracy_list = []
    wrong_purpose_count = 0
    
    # Loop through each row and print the data in the first and second columns
    for index, row in df.iterrows():
        prediction = predict(model, row['AccentlessSentences'])
        diff_count_1, diff_words_1 = compare_strings(row['AccentlessSentences'], row['Sentences'])
        diff_count_2, diff_words_2 = compare_strings(prediction, row['Sentences'])
        if diff_count_1 != 0:
            accuracy = (diff_count_1 - diff_count_2) / diff_count_1
            if accuracy < 0:
                accuracy=0
                wrong_purpose_count=wrong_purpose_count+1
        else:
            accuracy = 0
        print('Accuracy=', accuracy, '; Wrong purpose=', wrong_purpose_count) 
        print(diff_count_1, diff_words_1)
        print(diff_count_2, diff_words_2)
        accuracy_list.append(accuracy)
        
    # Calculate the average accuracy
    avg_accuracy = sum(accuracy_list) / len(accuracy_list)

    print('Accuracy=', avg_accuracy)

# Call the function with your CSV file
print_columns('myDataset/testDataset.csv')


In [6]:
pd.read_csv('myDataset/testDataset.csv')

,Sentences,AccentlessSentences
0,Chương trình này là hoạt động đặc biệt của Mob...,Chưong trinh này la hoat dộng dặc biet cua Mob...
1,"Mang tính thiết thực cao, chương trình được đá...","Mang tinh thiet thuc cao, chương trinh đuoc da..."
2,"Chia sẻ với PV trong buổi lễ công bố, nhiều cô...","Chia se voi PV trong buổi lễ cong bo, nhieu cô..."
3,Năm nay thì khác rồi.,Năm nay thi khác roi.
4,Cảm kích nhất là cảm giác được đồng cảm và chi...,Cam kích nhat la cảm giác duợc dong cam và chi...
...,...,...
4995,Để giúp CNVC-LĐ và nhân dân 6 tỉnh Miền Trung ...,Dể giup CNVC-LD va nhan dân 6 tinh Mien Trung ...
4996,"Tại TP Hồ Chí Minh: 43 Hoa Đào, quận Phú Nhuận...","Tai TP Ho Chí Minh: 43 Hoa Đao, quan Phu Nhuan..."
4997,"Báo Lao Động, Quỹ TLV Lao Động chân thành cảm ...","Bao Lao Dộng, Quy TLV Lao Đong chan thành cam ..."
4998,* LĐLĐ TP.,* LDLĐ TP.
